# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [2]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [3]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-27-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-07-events.csv', '/home/workspace/event_data/2018-11-09-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-06-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-21-events.csv', '/home/workspace/event_data/2018-11-10-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-02-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [6]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [7]:
# Create a Keyspace 
try :
    session.execute ("""
    CREATE KEYSPACE IF NOT EXISTS song_app
        WITH REPLICATION =
        { 'class': 'SimpleStrategy', 'replication_factor' : 1}
    """)
except Exception as e:
    print("Exception while creating keyspace.."+e)

#### Set Keyspace

In [8]:
# Set KEYSPACE to the keyspace specified above
session.set_keyspace("song_app")

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [9]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
createQuery = "CREATE TABLE IF NOT EXISTS song_details_for_session"
createQuery = createQuery + "(sessionId int, itemInSession int, artist text, song text, length float, PRIMARY KEY (sessionId, itemInSession))"
try :
    session.execute(createQuery)
except Exception as e:
    print("Exception while creating keyspace.."+e)

In [11]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
def insert_data(queryType, session) :

    file = 'event_datafile_new.csv'

    with open(file, encoding = 'utf8') as f:
        csvreader = csv.reader(f)
        next(csvreader) # skip header
        try: 
            for line in csvreader:
                if queryType == "song_details_for_session" :
                    query = "INSERT INTO song_details_for_session (sessionId, itemInSession, artist, song, length)"
                    query = query + "VALUES (%s, %s, %s, %s, %s)"
                    session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))
                elif queryType == "song_details_for_user" :
                    query = "INSERT INTO song_details_for_user (sessionId, userId, itemInSession, artist, song, firstName, lastName)"
                    query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
                    session.execute(query, (int(line[8]), int(line[10]), int(line[3]), line[0], line[9], line[1], line[4]))
                elif queryType == "user_details_for_song" :
                    query = "INSERT INTO user_details_for_song (song, userId, firstName, lastName)"
                    query = query + "VALUES (%s, %s, %s, %s)"
                    session.execute(query, (line[9], int(line[10]), line[1], line[4]))
        except Exception as e:
            print("Exception while inserting to "+ queryType +" table.."+e)

In [12]:
insert_data("song_details_for_session", session)

#### Do a SELECT to verify that the data have been inserted into each table

In [13]:
## SELECT statement to verify the data was entered into the table
selectBySessionIdAndItemInSessionQuery = "SELECT artist, song, length from song_details_for_session WHERE sessionId = 338 AND itemInSession=4"
try :
    rows = session.execute(selectBySessionIdAndItemInSessionQuery)
except Exception as e:
    print("Exception while querying by sessionId and itemInSession.."+e)
    
for row in rows :
    print("Artist ==> {0}, Song ==> {1}, Length ==> {2}".format(row.artist, row.song, row.length))

Artist ==> Faithless, Song ==> Music Matters (Mark Knight Dub), Length ==> 495.30731201171875


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [14]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
createQuery = """CREATE TABLE IF NOT EXISTS song_details_for_user
(sessionId int,
userId int,
itemInSession int,
artist text,
song text,
firstName text,
lastName text,
PRIMARY KEY ((userId, sessionId), itemInSession));"""

try :
    session.execute(createQuery)
except Exception as e:
    print("Exception while creating song_details_for_user table.."+e)

In [15]:
# Insert data into table for query 2
insert_data("song_details_for_user", session)
       

SyntaxError: unexpected EOF while parsing (<ipython-input-15-b6d8f5c77f79>, line 7)

In [29]:
## SELECT statement to verify the data was entered into the table
selectByUserIdAndSessionIdQuery = "SELECT artist, song, firstName, lastName FROM song_details_for_user WHERE userId=10 AND sessionId=182"
try :
    rows = session.execute(selectByUserIdAndSessionIdQuery)
except Exception as e:
    print("Exception while queying data from song_details_for_user table.."+e) 
    
for row in rows:
    print("Artist ==> {0}, Song ==> {1}, First Name ==> {2}, Last Name ==> {3}"
          .format(row.artist, row.song, row.firstname, row.lastname))

Artist ==> Down To The Bone, Song ==> Keep On Keepin' On, First Name ==> Sylvie, Last Name ==> Cruz
Artist ==> Three Drives, Song ==> Greece 2000, First Name ==> Sylvie, Last Name ==> Cruz
Artist ==> Sebastien Tellier, Song ==> Kilometer, First Name ==> Sylvie, Last Name ==> Cruz
Artist ==> Lonnie Gordon, Song ==> Catch You Baby (Steve Pitron & Max Sanna Radio Edit), First Name ==> Sylvie, Last Name ==> Cruz


In [30]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
createQuery = "CREATE TABLE IF NOT EXISTS user_details_for_song "
createQuery = createQuery + "(song text, userId int, firstName text, lastName text, PRIMARY KEY (song, userId))"
try :
    session.execute(createQuery)
except Exception as e:
    print("Exception while creating user_details_for_song table.."+e)

In [31]:
# Insert data into table for query 3
insert_data("user_details_for_song", session)

In [32]:
## SELECT statement to verify the data was entered into the table
query = "SELECT firstName, lastName FROM user_details_for_song where song = 'All Hands Against His Own'"
rows = session.execute(query)
for row in rows:
    print("First Name ==> {0}, Last Name ==> {1}".format(row.firstname, row.lastname))

First Name ==> Jacqueline, Last Name ==> Lynch
First Name ==> Tegan, Last Name ==> Levine
First Name ==> Sara, Last Name ==> Johnson


### Drop the tables before closing out the sessions

In [1]:
## Drop the table before closing out the sessions
session.execute("drop table song_details_for_session")
session.execute("drop table song_details_for_user")
session.execute("drop table user_details_for_song")

NameError: name 'session' is not defined

### Close the session and cluster connection¶

In [35]:
session.shutdown()
cluster.shutdown()